In [1]:
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain import hub
from langchain_openai import ChatOpenAI
from composio_langchain import ComposioToolSet, Action, App
llm = ChatOpenAI()
prompt = hub.pull("hwchase17/openai-functions-agent")

composio_toolset = ComposioToolSet(api_key="kc73e496wa2vqwzthdtta")
tools = composio_toolset.get_tools(actions=['GMAIL_SEND_EMAIL'])

In [2]:
from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.tools.reasoning import ReasoningTools

email_writing_agent = Agent(
    name="Email Writing Assistant",
    role="Compose professional, context-appropriate emails based on given inputs",
    model=OpenAIChat("gpt-4.1-nano"),
    tools=[ReasoningTools(add_instructions=True)],
    instructions=[
            """
    **Email Drafting Rules:**
    1. **NEVER use placeholders** like `[Name]`, `[Your Position]`, or `[...]`.  
       - If a detail is missing, **skip it entirely**.
    2. **Signature must be short and fixed**:  
       - Use **"Thanks,"** (if casual) or **"Best regards,"** (if formal).  
       - **No name, title, or company** unless explicitly provided.
    3. **Body must be complete**: No `[point 1]`, `[date]`, etc.
    """
    ],
    markdown=True # Since we want the actual email text, not JSON
)

In [3]:
response = email_writing_agent.run("""  
                                 Draft mail to : r2raviteja@gmail.com;
                                    Successfully applied to position and attached Cover Letter. 
                                   """)
email_writing_agent.print_response(response.content)

Output()

In [4]:
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
task = " mail to r2raviteja@gmail.com; attach file : CL.txt "+response.content

result = agent_executor.invoke({"input": task})
print(result)



> Entering new AgentExecutor chain...

Invoking: `GMAIL_SEND_EMAIL` with `{'attachment': 'CL.txt', 'body': 'Dear Raviteja,\n\nI am pleased to inform you that I have successfully applied for the position. Please find the job description attached for your reference.\n\nThank you for your support.\n\nThanks,', 'recipient_email': 'r2raviteja@gmail.com', 'subject': 'Application Submission and Job Description'}`


{'data': {'response_data': {'id': '196874c3047f6b73', 'labelIds': ['SENT'], 'threadId': '196874c3047f6b73'}}, 'error': None, 'successfull': True, 'successful': True}The email has been sent successfully to r2raviteja@gmail.com.

> Finished chain.
{'input': ' mail to r2raviteja@gmail.com; attach file : CL.txt Subject: Application Submission and Job Description\n\nDear Raviteja,\n\nI am pleased to inform you that I have successfully applied for the position. Please find the job description attached for your reference.\n\nThank you for your support.\n\nThanks,', 'output': 'The email 

In [4]:
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain import hub
from langchain_openai import ChatOpenAI
from composio_langchain import ComposioToolSet, Action, App
llm = ChatOpenAI()
prompt = hub.pull("hwchase17/openai-functions-agent")

composio_toolset = ComposioToolSet(api_key="kc73e496wa2vqwzthdtta")
tools = composio_toolset.get_tools(actions=['GMAIL_SEND_EMAIL'])

import pandas as pd
df = pd.read_csv(r"JobsData.csv")
num_job = 3

job_url = df["Apply_URL"].iloc[num_job]
with open("CL.docx", "w", encoding="utf-8") as f:
    f.write(df["Coverletter"].iloc[num_job])




In [9]:
JD = df["Combined"].iloc[num_job]

In [24]:
from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.tools.reasoning import ReasoningTools

# Updated Recruiter Email Agent
recruiter_email_agent = Agent(
    name="Recruiter Email Agent",
    role="Compose a concise follow-up email to a recruiter after submitting job application.",
    model=OpenAIChat("gpt-4.1-nano", temperature=0.3),
    tools=[ReasoningTools(add_instructions=True)],
    instructions=[
        """
        **Strict Rules:**
        1. NEVER use placeholders like [Name] or [Company]. Write generically if information is missing.
        2. NEVER include bullet points, numbered lists, or markdown in the email body.
        3. Keep the email to 2 short paragraphs maximum (3-4 sentences each).
        4. Mention the cover letter ONLY in the closing section.

        **Required Elements:**
        - State you've applied for the position
        - Highlight one specific qualification from the job description
        - Reference the cover letter at the end before your signature

        **Email Structure:**
        - Subject Line: "Follow-Up on [Job Title] Application" (use actual title or "the position")
        - Body:
          * Paragraph 1: Confirm your application and express enthusiasm about one key requirement from the JD
          * Paragraph 2: Offer additional value (portfolio/work sample if relevant) and invite discussion
        - Closing: Mention cover letter, then professional sign-off

        **Example Flow:**
        "I recently applied for [role] at [company]..."
        "My experience in [JD requirement] makes me particularly excited about..."
        "As detailed in my attached cover letter, I would bring [value]..."
        "I'd welcome the opportunity to discuss..."

        **Style:**
        - Professional but warm tone
        - Focus on value you bring (not just features)
        - Cover letter reference should be brief and natural

        **Output Format:**
        Return ONLY the complete email text ready to send:
        
        Subject: Follow-Up on [Job Title] Application
        
        Dear [Recruiter's Name or Hiring Team],
        
        [Paragraph 1: Application confirmation + 1 key qualification]
        [Paragraph 2: Value add + call to action]
        
        As mentioned in my attached cover letter, [brief reinforcement]. 
        
        Best regards,
        [Your Name]
        """
    ],
    markdown=True
)

response = recruiter_email_agent.run(f"{JD}")
recruiter_email_agent.print_response(response.content)

Output()

In [22]:
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
task = " mail to r2raviteja@gmail.com; attach file : CL.docx "+response.content

result = agent_executor.invoke({"input": task})
print(result)



> Entering new AgentExecutor chain...

Invoking: `GMAIL_SEND_EMAIL` with `{'attachment': 'CL.docx', 'body': 'Dear Hiring Team,\n\nI am excited about the opportunity to contribute to Goodparty.org as a Data Scientist, leveraging my expertise in Tableau dashboard development and data transformation to support your mission of empowering independents and transforming civic leadership. My experience aligns well with your needs for creating insightful, user-friendly dashboards that support organizational goals.\n\nI would be happy to share my portfolio or work samples demonstrating my skills in data visualization and dashboard design. Thank you for considering my application; I am available at your convenience for further discussion and look forward to the possibility of contributing to your impactful work.\n\nBest regards,\nRaviteja', 'recipient_email': 'r2raviteja@gmail.com', 'subject': 'Follow-Up on Data Scientist Application'}`


{'data': {'response_data': {'id': '19694c29d602eb9e', 'l